<a href="https://colab.research.google.com/github/rpmuller/boys/blob/main/boys_speed_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Compare the speed of different methods of computing the `erf` and `erfc` functions.

References:
- https://pubs.aip.org/aip/jcp/article/155/17/174117/565590/A-fast-algorithm-for-computing-the-Boys-function
- https://chemistry.stackexchange.com/questions/41214/boys-function-for-gaussian-integrals-in-ab-initio-calculations
- https://link.springer.com/article/10.1007/s10910-016-0668-x
- https://onlinelibrary.wiley.com/doi/full/10.1002/jcc.23935
- https://link.springer.com/article/10.1007/s10910-005-9023-3
- https://github.com/evaleev/libint/blob/master/include/libint2/boys.h


Code from MolecularIntegrals.jl:

In [ ]:
import Pkg
Pkg.add("SpecialFunctions")
Pkg.add("BenchmarkTools")
Pkg.add("OffsetArrays")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
    Updating `~/.julia/environments/v1.10/Project.toml`
  [6fe1bfb0] + OffsetArrays v1.15.0
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [ ]:
using SpecialFunctions
using BenchmarkTools
using OffsetArrays

In [ ]:
n = 1
xrange = range(1e-15, 15, length=10000)

1.0e-15:0.0015001500150015:15.0

In [ ]:
# Incomplete gamma function implementation.
boys2(n, x) = gamma(0.5 + n) * gamma_inc(0.5 + n, x, 0)[1] / (2x^(0.5 + n))

boys2 (generic function with 1 method)

In [ ]:
@benchmark boys2.(n, xrange)

BenchmarkTools.Trial: 1972 samples with 1 evaluation per sample.
 Range (min … max):  1.814 ms …   7.132 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.601 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.521 ms ± 477.964 μs  ┊ GC (mean ± σ):  0.07% ± 1.19%

  ▆▅▂               █▇▇▆▃▂▁                                   ▁
  ████▇▆▅▄▆▅▄▆▆▆▁▄▁█████████▆▄▆▇▆▆▆▅▁▁▄▄▄▅▁▁▅▁▁▁▄▅▆▅▆▅▄▁▁▁▄▁▅ █
  1.81 ms      Histogram: log(frequency) by time      4.24 ms <

 Memory estimate: 78.23 KiB, allocs estimate: 3.

In [ ]:
function Fm(m,T,SMALL=1e-18)
    mhalf = m+0.5
    #T = max(T,SMALL) # Underflow; is there a better way to fix this?
    if T<SMALL return 1/(2m+1) end # This seems to work??
    return gamma(mhalf)*gamma_inc(mhalf,T,0)[1]/(2T^(mhalf))
end

Fm (generic function with 2 methods)

In [ ]:
@benchmark Fm.(n,xrange)

BenchmarkTools.Trial: 2332 samples with 1 evaluation per sample.
 Range (min … max):  1.793 ms …   6.740 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.870 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.134 ms ± 426.739 μs  ┊ GC (mean ± σ):  0.08% ± 1.27%

   █▃                                                          
  ████▅▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▄▇▅▄▄▅▃▃▂▂▂▂▂▂▂▁▂▂▂▂▂▁▁▂▂▁▂ ▃
  1.79 ms         Histogram: frequency by time        3.18 ms <

 Memory estimate: 78.23 KiB, allocs estimate: 3.

In [ ]:
@benchmark Fm_ref.(n,xrange)

BenchmarkTools.Trial: 2432 samples with 1 evaluation per sample.
 Range (min … max):  1.840 ms …   6.064 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.918 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.048 ms ± 250.496 μs  ┊ GC (mean ± σ):  0.07% ± 1.23%

   ▇█                       ▁                                  
  ▃███▇▇▄▄▃▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▅██▆▅▄▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▁▁▁▂▁▂▂▂▂▁▂▁▁▂ ▃
  1.84 ms         Histogram: frequency by time        2.71 ms <

 Memory estimate: 78.23 KiB, allocs estimate: 3.

# Libint suite of methods

What follows is as close to a transcription of [what libint does](https://github.com/evaleev/libint/blob/master/include/libint2/boys.h). Experience shows that the combination of the techniques they use is very fast, but I think that's the functions at the end.

`Fm_ref(m,T)` - Libint reference implementation of Boys function `Fm`
 $$ F_m (T) = \int_0^1 u^{2m} \exp(-T u^2)du, $$
using multi-algorithm approach (upward recursion for T>=117, and
asymptotic summation for T<117) in conjunction with Fm_recur_array, below.

This is slow and should be used for reference purposes, e.g. computing the interpolation tables.

In [ ]:
function Fm_ref(m,T,eps = 1e-10)
    denom = (m + 0.5)
    term = exp(-T) /2denom
    old_term = 0.0
    sum = term
    while term > sum*eps || old_term < term # They use a get_epsilon function and then compare to epsilon/10 for some reason.
        denom += 1
        old_term = term
        term = old_term * T / denom
        sum += term
    end
    return sum
end

Fm_ref_array(mmax,T) = Fm_ref.(0:mmax,T)

Fm_ref_array (generic function with 1 method)

In [ ]:
function Fm_recur_array(mmax,T)
  K = 0.5*sqrt(pi)
  T2 = 2T
  eT = exp(-5)
  sqrt_T = sqrt(T)
  Fm = OffsetArray(zeros(Float64,mmax+1),0:mmax)
  Fm[0] = K*erf(sqrt_T)/sqrt_T
  for m in 0:(mmax-1)
    Fm[m+1] = ((2*m+1)*Fm[m]-eT)/T2
  end
  return Fm
end

Fm_recur_array (generic function with 1 method)

In [ ]:
function Fm_ref2_array(mmax,T)
  return T < 117 ? Fm_ref_array(mmax,T) : Fm_recur_array(mmax,T)
end

Fm_ref2_array (generic function with 1 method)